In [1]:
!pip install pypdf2 llama-index llama-index-embeddings-huggingface llama-index-llms-huggingface llama-index-llms-huggingface-api

In [3]:
from pypdf import PdfReader
import os
from transformers import AutoModelForCausalLM, AutoTokenizer  # (필요시 로컬 LLM 대안용)
from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.node_parser import SentenceSplitter
# OpenAI 임베딩 모델 임포트
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor
from pypdf import PdfReader
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

In [5]:
llm = OpenAI(
        model="gpt-4",         # 사용할 OpenAI 모델명, 필요에 따라 변경 (예: gpt-3.5-turbo)
        temperature=0.01,
        max_tokens=512,
        api_key='여러분들의 챗지피티 api 키',
        system_prompt=(
            "당신은 주어진 문서 내용만 기반으로 답변하는 AI 어시스턴트입니다. "
            "검색된 문서 조각들 내용을 활용하여 질문에 답변하세요. "
            "검색된 문서에 관련 정보가 있다면 반드시 그 내용을 기반으로 답변을 생성하세요. "
            "문서에 명확히 언급된 내용을 우선으로 답변하고, 문서에 있는 내용만 참고하여 한국어로 명확하고 정확하게 답변해주세요. "
            "답변을 생성할 때 관련 문서 내용을 직접 인용하고, 문서에 없는 내용은 추가하지 마세요."
        )
    )

In [6]:
embed_model = OpenAIEmbedding(
        model="text-embedding-3-small",  # OpenAI의 최신 임베딩 모델 사용
        api_key='여러분들의 챗지피티 api 키',
        embed_batch_size=10,  # 배치 처리 크기 설정
        dimensions=1536  # 임베딩 벡터 차원 수
    )

In [8]:
import os
os.chdir('/content/drive/MyDrive/data')

In [9]:
def process_pdf_and_create_index(pdf_path):
    """PDF 파일을 처리하고 인덱스를 생성하는 함수"""
    # PDF 파일 로드 및 텍스트 추출
    pdf_reader = PdfReader(pdf_path)
    text_chunks = []

    # 모든 페이지의 텍스트 추출
    for page_num, page in enumerate(pdf_reader.pages):
        text = page.extract_text()
        if text.strip():  # 빈 페이지 제외
            # 페이지 번호 정보 포함
            text_with_metadata = f"[페이지 {page_num + 1}] {text}"
            text_chunks.append(text_with_metadata)

    # LlamaIndex Document 객체 생성
    documents = [Document(text=chunk) for chunk in text_chunks]

    # 문서 분할 (청킹)
    node_parser = SentenceSplitter(
        chunk_size=512,       # 청크 크기
        chunk_overlap=50,     # 오버랩
        paragraph_separator="\n\n",
        secondary_chunking_regex="(?<=\. )",
        include_metadata=True,
        include_prev_next_rel=True  # 이전/다음 청크 관계 포함
    )


    # Settings 설정
    Settings.llm = llm
    Settings.embed_model = embed_model
    Settings.node_parser = node_parser

    # 노드 파싱
    nodes = node_parser.get_nodes_from_documents(documents)

    # 인덱스 생성
    index = VectorStoreIndex(nodes)

    # 인덱스 저장
    index_dir = "pdf_index_storage"
    os.makedirs(index_dir, exist_ok=True)
    index.storage_context.persist(persist_dir=index_dir)

    return index


def load_saved_index():
    """저장된 인덱스 로드"""
    index_dir = "pdf_index_storage"
    if os.path.exists(index_dir):
        # 인덱스 로드
        storage_context = StorageContext.from_defaults(persist_dir=index_dir)
        index = load_index_from_storage(storage_context)
        return index
    return None

def create_optimized_query_engine(index):
    """최적화된 쿼리 엔진 생성"""
    # 검색기 설정 (Top-k 파라미터 조정)
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=5,  # 관련성 높은 문서 5개로 조정
        vector_store_query_mode="default"
    )

    # 유사도 임계값 설정
    node_postprocessors = [SimilarityPostprocessor(similarity_cutoff=0.2)]

    # 쿼리 엔진 생성
    query_engine = RetrieverQueryEngine.from_args(
        retriever=retriever,
        node_postprocessors=node_postprocessors,
        response_mode="compact",  # 모든 관련 문서를 한번에 고려
        response_kwargs={"verbose": True}  # 디버깅을 위한 verbose 모드
    )

    return query_engine

<>:23: SyntaxWarning: invalid escape sequence '\.'
<>:23: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipython-input-2899137503.py:23: SyntaxWarning: invalid escape sequence '\.'
  secondary_chunking_regex="(?<=\. )",


In [10]:
pdf_path = '/content/drive/MyDrive/data/선진기업복지 업무매뉴얼.pdf'

In [13]:
index = load_saved_index()

if index is None :
    index = process_pdf_and_create_index(pdf_path)

In [15]:
Settings.llm = llm
Settings.embed_model = embed_model

In [16]:
query_engine = create_optimized_query_engine(index)

In [22]:
# user_question = '사내근로복지기금의 회계 관리는 어떻게 이루어지나요?'
user_question = '컨설턴트는 공단 직원인가요?'


In [23]:
response = query_engine.query(user_question)

In [24]:
print( response.response )

아니요, 컨설턴트는 공단 직원이 아닙니다. 컨설턴트는 선진기업복지와 관련된 외부 전문가들로 구성되어 있습니다. 이들은 기본 자질과 능력을 검토한 후 '컨설턴트 양성 및 직무 교육'을 이수한 자에 한하여 공단에서 선발, 위촉됩니다.
